In [1]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/bus_terminals.csv')
merged_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/merged_matrix.csv')
bus_capacity = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/refs/heads/main/datasets/bus_divisions_cleaned_capacity%20(1).csv')

# VRP

In [2]:
# ============================================================
# 0)  CONFIG
# ============================================================
FILE = "new_matrix.csv"          # distance/time matrix (seconds)
TIME_LIMIT_SEC = 60           # give the solver 3 minutes
MAX_STOPS_PER_ROUTE = 9     # None = unlimited

# exact list & order of depots to use
TERMINALS = ['BD14', 'BD15', 'BT03', 'BL14', 'BD04', 'BT11', 'BT06', 'BL20', 'BT18', 'BL23']

# ============================================================
# 1)  LOAD & CLEAN THE MATRIX
# ============================================================
import pandas as pd, numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

df  = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/new_matrix.csv', index_col=0)                 # IDs in the index
bad = df.apply(pd.to_numeric, errors="coerce")   # force numeric

if bad.isna().any().any():
    r = bad.index[ bad.isna().any(axis=1) ][0]
    c = bad.columns[ bad.isna().any(axis=0) ][0]
    raise ValueError(f"non-numeric cell at row {r!r}, column {c!r}")

# Drop unwanted depots from the matrix
excluded_depots = {'BD16', 'BT24', 'BT08', 'BT16', 'BT21'}
df = df.drop(index=excluded_depots, columns=excluded_depots, errors='ignore')

dist_mat = df.astype(np.int32).to_numpy()       # seconds
ids      = df.index.astype(str).tolist()
n_nodes  = len(ids)

# ------------------------------------------------------------
# make sure every requested depot exists in the CSV
missing = [d for d in TERMINALS if d not in ids]
assert not missing, f"Depot(s) not found in CSV: {missing}"

# OR-Tools wants the depot indices
bus_idx = [ids.index(d) for d in TERMINALS if d in ids]

# classify the other nodes (not strictly needed, but sanity)
acc_idx   = [i for i,s in enumerate(ids) if s.startswith("A")]
venue_idx = [i for i,s in enumerate(ids) if s.startswith("V")]
assert acc_idx and venue_idx, "Need at least one A- and one V- node"

# ============================================================
# 2)  ROUTING MODEL
# ============================================================
man = pywrapcp.RoutingIndexManager(n_nodes, len(bus_idx), bus_idx, bus_idx)
rt  = pywrapcp.RoutingModel(man)

# transit (cost) = travel seconds
def sec_cb(fi, ti):
    f, t = man.IndexToNode(fi), man.IndexToNode(ti)
    return int(dist_mat[f][t])

transit = rt.RegisterTransitCallback(sec_cb)
rt.SetArcCostEvaluatorOfAllVehicles(transit)

# limit total stops if desired
if MAX_STOPS_PER_ROUTE:
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, MAX_STOPS_PER_ROUTE, True, "Stops")
else:
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, 1000, True, "Stops")  # Arbitrary upper limit

# ---- force every bus to be used (≥ 1 real stop) ---------------------------
if MAX_STOPS_PER_ROUTE:
    stop_dim = rt.GetDimensionOrDie("Stops")
else:
    # create a tiny 1-per-hop dimension just for this purpose
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, 10**6, True, "MustUse")
    stop_dim = rt.GetDimensionOrDie("MustUse")

# ---- ensure at least one venue per route ---------------------------
venue_indices = [man.NodeToIndex(i) for i in venue_idx]
for vehicle_id in range(len(bus_idx)):
    # Add a constraint to ensure at least one venue is visited per route
    venue_visited = [rt.NextVar(venue) for venue in venue_indices]
    rt.solver().Add(rt.solver().Sum(venue_visited) > 1)

#--- set minimum stops per route ---------------------------
stops_dimension = rt.GetDimensionOrDie("Stops")
for vehicle_id in range(len(TERMINALS)):
    end_index = rt.End(vehicle_id)
    stops_dimension.CumulVar(end_index).SetMin(3)  # Set minimum useful length


# ============================================================
# 3)  SEARCH PARAMETERS
# ============================================================
p = pywrapcp.DefaultRoutingSearchParameters()
p.first_solution_strategy    = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
p.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
p.time_limit.seconds         = TIME_LIMIT_SEC

solution = rt.SolveWithParameters(p)

# ============================================================
# 4)  SINGLE-LINE REPORT  (seconds → minutes)
# ============================================================
if not solution:
    print("❌  No solution found — try increasing TIME_LIMIT_SEC "
          "or relaxing MAX_STOPS_PER_ROUTE.")
else:
    for v, depot_node in enumerate(bus_idx):
        idx = rt.Start(v)
        route = [ids[depot_node]]          # start depot
        time_sec = 0

        while not rt.IsEnd(idx):
            prev = idx
            idx  = solution.Value(rt.NextVar(idx))
            route.append(ids[man.IndexToNode(idx)])
            time_sec += dist_mat[man.IndexToNode(prev), man.IndexToNode(idx)]

        stops = len(route) - 2              # minus start & return depot
        minutes = time_sec / 60
        print(f"{ids[depot_node]:<5}: " + " → ".join(route)
              + f"   ({stops} stops, {minutes:.1f} min)")

BD14 : BD14 → A12 → V17 → A33 → A34 → A41 → A14 → V31 → A13 → BD14   (8 stops, 74.6 min)
BD15 : BD15 → A1 → A19 → A20 → A21 → A4 → A2 → BD15   (6 stops, 71.7 min)
BT03 : BT03 → A16 → V20 → A32 → A11 → A36 → A45 → A35 → BT03   (7 stops, 155.7 min)
BL14 : BL14 → V12 → A6 → A10 → A39 → A40 → V30 → A18 → A15 → BL14   (8 stops, 138.8 min)
BD04 : BD04 → A17 → V9 → BD04   (2 stops, 26.6 min)
BT11 : BT11 → A3 → V24 → V14 → A31 → BT11   (4 stops, 51.0 min)
BT06 : BT06 → A28 → A27 → V5 → A29 → A26 → A8 → BT06   (6 stops, 75.9 min)
BL20 : BL20 → A51 → V8 → A49 → A48 → A47 → A50 → V11 → A46 → BL20   (8 stops, 64.2 min)
BT18 : BT18 → A22 → V28 → A44 → A9 → A30 → A42 → A38 → A24 → BT18   (8 stops, 238.0 min)
BL23 : BL23 → A23 → A43 → A7 → V18 → A37 → V19 → A5 → A25 → BL23   (8 stops, 107.8 min)


### node_ids

In [3]:
# Extract the first column (index) from new_matrix
node_ids = df.index.tolist()

# Initialize an empty list to store the coordinates
coordinates = []

# Iterate through the node IDs and match them with the corresponding coordinates
for node_id in node_ids:
    if node_id in venues['id'].values:
        # Match with venues
        venue_row = venues[venues['id'] == node_id]
        coordinates.append([node_id, venue_row['Latitude'].values[0], venue_row['Longitude'].values[0]])
    elif node_id in accomodations_clusters['id'].values:
        # Match with accommodation clusters
        accom_row = accomodations_clusters[accomodations_clusters['id'] == node_id]
        coordinates.append([node_id, accom_row['avg_latitude'].values[0], accom_row['avg_longitude'].values[0]])
    elif node_id in bus_terminals['id'].values:
        # Match with bus terminals
        terminal_row = bus_terminals[bus_terminals['id'] == node_id]
        coordinates.append([node_id, terminal_row['Latitude'].values[0], terminal_row['Longitude'].values[0]])
    else:
        # If no match is found, append NaN for coordinates
        coordinates.append([node_id, float('nan'), float('nan')])

# Create the node_coords dataframe
node_coords = pd.DataFrame(coordinates, columns=['Node ID', 'Latitude', 'Longitude'])

# Display the resulting dataframe
node_coords

,Node ID,Latitude,Longitude
0,BD14,33.993948,-118.476437
1,BD15,34.085197,-118.382010
2,BT03,34.201350,-118.450750
3,BL14,33.901888,-118.285809
4,BD04,33.776882,-118.202795
...,...,...,...
70,A47,34.091356,-118.279988
71,A48,34.063247,-118.298355
72,A49,34.054311,-118.280299
73,A50,34.083647,-118.255568


In [4]:
# Add a new column 'Type' to node_coords based on the first letter of 'Node ID'
node_coords['Type'] = node_coords['Node ID'].apply(
    lambda x: 'Depot' if x.startswith('B') else 
              'Accommodation' if x.startswith('A') else 
              'Venue' if x.startswith('V') else 'Unknown'
)

# Display the updated dataframe
node_coords

,Node ID,Latitude,Longitude,Type
0,BD14,33.993948,-118.476437,Depot
1,BD15,34.085197,-118.382010,Depot
2,BT03,34.201350,-118.450750,Depot
3,BL14,33.901888,-118.285809,Depot
4,BD04,33.776882,-118.202795,Depot
...,...,...,...,...
70,A47,34.091356,-118.279988,Accommodation
71,A48,34.063247,-118.298355,Accommodation
72,A49,34.054311,-118.280299,Accommodation
73,A50,34.083647,-118.255568,Accommodation


In [5]:
node_ids = node_coords['Node ID'].tolist()
node_id_to_coords = dict(zip(node_coords['Node ID'], zip(node_coords['Latitude'], node_coords['Longitude'])))
node_id_to_type = dict(zip(node_coords['Node ID'], node_coords['Type']))


### map!

In [6]:
import folium
from folium import PolyLine


# Map center around LA
la_center = [34.0522, -118.2437]
m = folium.Map(location=la_center, zoom_start=11)

# Color mapping for node types
type_colors = {
    'Depot': 'red',
    'Venue': 'green',
    'Accommodation': 'blue'
}

# Emoji mapping for tooltip fun
type_emojis = {
    'Depot': '🚌',
    'Venue': '🏟️',
    'Accommodation': '🏨'
}

# Add markers for all nodes
for node_id in node_ids:
    coord = node_id_to_coords.get(node_id)
    if coord is None:
        continue  # skip if coordinates missing
    
    node_type = node_id_to_type.get(node_id, 'Accommodation')  # default to Accommodation
    color = type_colors.get(node_type, 'gray')
    emoji = type_emojis.get(node_type, '')
    
    folium.CircleMarker(
        location=coord,
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.9,
        tooltip=f"{emoji} {node_id} ({node_type})"
    ).add_to(m)

# Colors for routes
route_colors = ['purple', 'orange', 'darkred', 'cadetblue', 'darkblue', 'blue', 'gray', 'darkgreen']

# Plot the routes from your solver solution
if solution:
    for v, depot_node in enumerate(bus_idx):
        idx = rt.Start(v)
        route_coords = []
        
        while not rt.IsEnd(idx):
            node = man.IndexToNode(idx)
            node_id = ids[node]
            coord = node_id_to_coords.get(node_id)
            if coord:
                route_coords.append(coord)
            idx = solution.Value(rt.NextVar(idx))
            
        # Add last depot coord to close the route
        end_node = man.IndexToNode(idx)
        end_node_id = ids[end_node]
        end_coord = node_id_to_coords.get(end_node_id)
        if end_coord:
            route_coords.append(end_coord)
        
        # Draw polyline for route if it has at least 2 points
        if len(route_coords) > 1:
            folium.PolyLine(
                route_coords,
                color=route_colors[v % len(route_colors)],
                weight=5,
                opacity=0.7,
                tooltip=f"Bus route {v+1} (Depot {ids[depot_node]})"
            ).add_to(m)
else:
    print("No solution found to plot.")


m

/Users/leonardogreco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
metro = pd.read_csv('/Users/leonardogreco/Documents/EPFL/M2/Operations/SLO_LA_Olympics/datasets/Metro.csv')
# Create a map centered around Los Angeles
# Add markers for each metro station
for _, row in metro.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        fill_opacity=0.9,
        popup=f"ID: {row['stop_id']}<br>Stop: {row['stop_name']}",
        color='magenta',
        fill_color='cyan',
    ).add_to(m)
# Display the map
m


In [8]:
# Extract nodes from bus route 3
route_3_nodes = []
bus_index = 2  # Bus route 3 corresponds to index 2

if solution:
    idx = rt.Start(bus_index)
    while not rt.IsEnd(idx):
        node = man.IndexToNode(idx)
        route_3_nodes.append(ids[node])
        idx = solution.Value(rt.NextVar(idx))
    # Add the last depot node to close the route
    route_3_nodes.append(ids[man.IndexToNode(idx)])

route_3_nodes

['BT03', 'A16', 'V20', 'A32', 'A11', 'A36', 'A45', 'A35', 'BT03']

In [9]:
# Extract nodes from bus route 5
route_5_nodes = []
bus_index = 4  # Bus route 5 corresponds to index 4

if solution:
    idx = rt.Start(bus_index)
    while not rt.IsEnd(idx):
        node = man.IndexToNode(idx)
        route_5_nodes.append(ids[node])
        idx = solution.Value(rt.NextVar(idx))
    # Add the last depot node to close the route
    route_5_nodes.append(ids[man.IndexToNode(idx)])

route_5_nodes

['BD04', 'A17', 'V9', 'BD04']

## Analysis of inital VRP

### CO2 Impact estimation

In [10]:
# Calculate the total time for all routes
total_route_time = 0

if solution:
    for v, depot_node in enumerate(bus_idx):
        idx = rt.Start(v)
        route_time = 0
        
        while not rt.IsEnd(idx):
            prev = idx
            idx = solution.Value(rt.NextVar(idx))
            route_time += dist_mat[man.IndexToNode(prev), man.IndexToNode(idx)]
        
        total_route_time += route_time

# Convert total time to minutes
total_route_time_minutes = total_route_time / 60
print(f"Total Route Time: {total_route_time_minutes:.2f} minutes")

Total Route Time: 1004.08 minutes


In [11]:
# Constants for CO₂ emissions (in grams CO₂ per km per bus)
EMISSION_DIESEL = 1050   # baseline for traditional buses (grams/km)
EMISSION_ELECTRIC = 0    # assuming zero emissions for electric buses

# Average speed of a bus in km/h
AVERAGE_SPEED_KMPH = 25  # adjust based on traffic assumptions in LA

# Assuming route_times is a list of route durations in hours
# e.g., [1.2, 0.75, 0.5] meaning 1.2h, 45min, 30min
total_time_hours = total_route_time_minutes / 60.

# Convert total time to estimated total distance using average speed
total_distance = total_time_hours * AVERAGE_SPEED_KMPH

# CO₂ Emissions
co2_emissions_diesel = total_distance * EMISSION_DIESEL
co2_emissions_electric = total_distance * EMISSION_ELECTRIC
co2_saved = co2_emissions_diesel - co2_emissions_electric

# Output
print(f"Total Time Traveled: {total_time_hours:.2f} hours")
print(f"Estimated Total Distance: {total_distance:.2f} km (assuming {AVERAGE_SPEED_KMPH} km/h)")
print(f"CO₂ Emissions with Diesel Buses: {co2_emissions_diesel/1000:.2f} kg")
print(f"CO₂ Emissions with Electric Buses: {co2_emissions_electric:.2f} g")
print(f"CO₂ Saved: {co2_saved/1000:.2f} kg")


Total Time Traveled: 16.73 hours
Estimated Total Distance: 418.37 km (assuming 25 km/h)
CO₂ Emissions with Diesel Buses: 439.29 kg
CO₂ Emissions with Electric Buses: 0.00 g
CO₂ Saved: 439.29 kg
